# Evaluate ensemble members

The goal of this script is to identify ensemble members that meet specific ecological criteria

In [284]:
import pandas as pd
import numpy as np
import os
import esm_tools
import importlib
importlib.reload(esm_tools)
pd.set_option('display.max_rows', 1000) 
import shutil
sys.path.append("/glade/u/home/adamhb/Earth-System-Model-Tools/process_output/")
import re
import fnmatch
import math
from matplotlib import pyplot as plt
pd.set_option('display.max_colwidth', None)

In [357]:
write_csv_of_passing_members = False
setup_new_simulation = True
check_CZ2_100_yrs = False
check_CZ2_PEAS = False
new_case_is_fire_suppression = True
new_run_is_hybrid = True
check_transient_vs_precolonial = False
check_CZ2_vs_STAN = False

new_subdirs_prefix = "CZ2_trans_110923"
harmonized_reference_case_name='CZ2_trans_110923_-17e2acb6a_FATES-8a054a12'
destination_run_dir='/glade/scratch/adamhb/archive/CZ2_trans_1950_2020_111023_-17e2acb6a_FATES-8a054a12/run'

if write_csv_of_passing_members == True:
    output_file_path = '/glade/scratch/adamhb/processed_output/CZ2_equilibrium_passing/CZ2-PEAS_passing_with_fire_110923.csv'

# Path to ensemble data to be evaluated
path_to_ensemble_data = '/glade/scratch/adamhb/processed_output/CZ2_equilibrium_passing/CZ2-PEAS_passing_110923.csv'

# Path to where ensemble parameter files are stored
path_to_ensemble_param_files_root = '/glade/u/home/adamhb/ahb_params/fates_api_25/ensembles/'

key_metrics = ['inst_id','BA','ShannonE','AGB','BA_pine','BA_cedar','BA_fir','BA_shrub','BA_oak',
'FailedPFTs','Pct_shrub_cover_canopy','Pct_shrub_cover','TreeStemD','TreeStemD_40','TreeStemD_60',
               'TreeStemD_80','TreeStemD_100','ResproutD_oak','ResproutD_shrub','NPP','Combustible_fuel']#,
#'Burned_area','Pct_high_severity_1700','Pct_high_severity_3500']

## Functions

In [324]:
def delete_all_files_in_directory(directory_path):
    try:
        # List all files in the directory
        file_list = os.listdir(directory_path)

        # Iterate over the files and delete them
        for file_name in file_list:
            file_path = os.path.join(directory_path, file_name)
            if os.path.isfile(file_path):
                os.remove(file_path)
        
        print(f"All files in {directory_path} have been deleted.")
    except Exception as e:
        print(f"An error occurred: {e}")


def get_most_recent_file_in_dir(folder_path):

    # Get a list of all files in the folder
    files = os.listdir(folder_path)

    # Filter out only files (excluding subdirectories)
    files = [file for file in files if os.path.isfile(os.path.join(folder_path, file))]

    # Sort the files by creation time (most recent first)
    files.sort(key=lambda x: os.path.getctime(os.path.join(folder_path, x)), reverse=True)

    # Check if there are any files in the folder
    if files:
        most_recent_file = files[0]
        print(f"The most recently created file is: {most_recent_file}")
        return most_recent_file
    else:
        print("The folder is empty.")

        

        
def aggregate_passing_paramsets(successfull_param_files,n_new_cases,n_inst_per_case,new_subdirs_prefix,
                                path_to_ensemble_param_files_root,new_param_file_base_name,fire_suppression = False,
                                nignitions = 0.055, fire_thresh = 1000):
    
    '''
    This function relabels and prepares parameter sets that passed criteria for a new group of cases to be run
    '''
    
    os.makedirs("tmp",exist_ok=True)
    delete_all_files_in_directory("tmp")
    
    # Make new subdirectories for the parameter files for the new cases
    for n in range(n_new_cases):
        new_param_subdir_suffix = str(n+1).rjust(2, '0')
        new_param_subdir = os.path.join(path_to_ensemble_param_files_root,new_subdirs_prefix) + "_" + new_param_subdir_suffix
        os.makedirs(new_param_subdir,exist_ok=True)
    
    
    # Put all successful param files in a temp direcotry
    tmp_inst = 0
    for reference_param_file in successfull_param_files:
        
        # Number them 0001 ... n
        tmp_inst = tmp_inst + 1
        
        # Get full path of originl param file
        ref_nc_file_full_path = reference_param_file
        print("originl_file:",ref_nc_file_full_path)

        #Copy to reference param file to new folder
        
        new_tag = str(tmp_inst).rjust(4, '0')
        new_name = new_param_file_base_name + "_" + new_tag + ".nc"
        dst_file = os.path.join("tmp",new_name)
        print("tmp file:",dst_file)
        shutil.copy(ref_nc_file_full_path,dst_file)
        
        # Generate new case tags for a new group of cases
        new_case_tags = [str(case_tag+1).rjust(2, '0') for case_tag in range(n_new_cases)]
         
    for i,new_case_tag in enumerate(new_case_tags):
            
            print("\n")
            
            # Make new subdir for the new cases
            #if len(new_case_tags) == 1:
            #    new_param_subdir = os.path.join(path_to_ensemble_param_files_root,new_subdirs_prefix)
            
            #else:
            new_param_subdir = os.path.join(path_to_ensemble_param_files_root,new_subdirs_prefix) + "_" + new_case_tag
            
            tmp_inst = i * n_inst_per_case
            
            for j in range(n_inst_per_case):
                
                tmp_inst = tmp_inst + 1
                tmp_inst_tag = str(tmp_inst).rjust(4, '0')
                
                 # Get temp param file with inst tag
                ref_nc_file = esm_tools.find_files_with_substring(directory="tmp",
                                                substring=tmp_inst_tag)

                # Get full path of originl param file
                ref_nc_file_full_path = os.path.join("tmp",ref_nc_file[0])
                
                print("tmp_file:",ref_nc_file_full_path)
                
                new_tag = str(j + 1).rjust(4, '0')
                new_name = new_param_file_base_name + "_" + new_tag + ".nc"
                dst_file = os.path.join(path_to_ensemble_param_files_root,new_param_subdir,new_name)
                
                shutil.copy(ref_nc_file_full_path,dst_file)
                
                
                # pick up here
                if new_case_is_fire_suppression == True:
                    esm_tools.assign_var_to_nc(dst_file,"fates_fire_nignitions",nignitions,0)
                    esm_tools.assign_var_to_nc(dst_file,"fates_fire_threshold",fire_thresh,0)
                
                
                print("destination:",dst_file)

def harmonic_mean(numbers):
    # Ensure the list is not empty and does not contain zero to avoid division by zero
    if not numbers or 0 in numbers:
        return None

    # Calculate the sum of the reciprocals of the numbers
    sum_of_reciprocals = sum(1 / x for x in numbers)
    
    # Calculate the harmonic mean
    return len(numbers) / sum_of_reciprocals

def root_mean_square(numbers):
    # Check if the list is empty to avoid division by zero
    if not numbers.all():
        return None
    
    # Calculate the squares and mean
    mean_of_squares = sum(x**2 for x in numbers) / len(numbers)
    
    # Return the square root of the mean of squares
    return math.sqrt(mean_of_squares)

## Load ensemble data

In [325]:
df = pd.read_csv(path_to_ensemble_data)
df = df.drop(df.columns[0], axis=1)

## Ecological expectations and preliminary filters

### Check on coexistence and extreme shrub dominance at 100 years

In [326]:
if check_CZ2_100_yrs == True:
    
    # Filter criteria
    filter_at_100_yrs = (df['FailedPFTs'] == 0) & (df['Pct_shrub_cover'] > 0.01) & (df['Pct_shrub_cover_canopy'] < 0.65)
    
    
    number_passing = len(df[filter_at_100_yrs])
    print(number_passing, "pass check on coexistence and shrub dominance")
    print(number_passing / len(df) * 100,"percent success")
    df_passing_CZ2_100_yrs = df[filter_at_100_yrs].sort_values("ShannonE",ascending = False)
    if write_csv_of_passing_members == True:
        df_passing_CZ2_100_yrs.to_csv(output_file_path)
        print("Wrote file:",output_file_path)
    #df_passing_CZ2_100_yrs[key_metrics]
    
    
    if setup_new_simulation == True:
        aggregate_passing_paramsets(successfull_param_files = list(df_passing_CZ2_100_yrs['param_file_path']),
                                n_new_cases = 7,
                                n_inst_per_case = 72,
                                new_subdirs_prefix = "CZ2_equilibrium_110523", # Without underscore
                                path_to_ensemble_param_files_root = path_to_ensemble_param_files_root,
                                new_param_file_base_name = 'CZ2_equilibrium_110523') # Without underscore

### CZ2, Pre-Euro-American Management, equilibrium

In [339]:
if check_CZ2_PEAS == True:

    # Burned area
    if 'Burned_area' in key_metrics:
        ba_dry = (df['Burned_area'] > 0.0294) & (df['Pct_high_severity_1700'] < 0.0909) # Williams et al., 2023
        print(sum(ba_dry),"are within range of observations for burned area")

        # Percent high severity
        phs_dry = (df['Pct_high_severity_1700'] > 1) & (df['Pct_high_severity_1700'] < 6)
        print(sum(phs_dry),"are within obs for PHS 1700 kW m-1")
        phs_dry = (df['Pct_high_severity_3500'] > 1) & (df['Pct_high_severity_3500'] < 6)
        print(sum(phs_dry),"are within obs for PHS 3500 kW m-1")

    # Shrub cover
    #shrub_cov = (df['Pct_shrub_cover'] > 0.14) & (df['Pct_shrub_cover'] < 0.32)
    #print(sum(shrub_cov),"are within range of obs for shrub cover")

    # Pine significant
    #df["pine_frac"] = df["BA_pine"] / df["BA"]

    #pine_frac_thresh = 0.10
    #pine_sig = df['pine_frac'] > pine_frac_thresh
    #print(sum(pine_sig),'have pine basal area >', pine_frac_thresh)

    # All metrics
    #if 'Burned_area' in key_metrics:
    #    all_metrics = len(df.loc[ba_dry & phs_dry & shrub_cov & pine_sig])
    #    print(all_metrics,'meet all expectations for CZ2 PEAS')
    
    # Sort by failed pfts and Shannon E
    df = df.sort_values(["FailedPFTs","ShannonE"],ascending=[True,False])
    
    
    # Score each param set based on structural variables
    #########
    #Targets#
    #########
    
    BA_target = 20
    BA_pine_target = 12
    BA_cedar_target = 6
    BA_fir_target = 1
    BA_oak_target = 3 # This is assumed
    Stem_den_target = 144 # > 10 cm, Safford and Stevens, 2017
    Stem_den_40_target = 65
    Stem_den_60_target = 43
    Stem_den_80_target = 29
    Stem_den_100_target = 17
    Shrub_cov_target = 0.23
    
    targets = [20,12,6,1,3,144,65,43,29,17,0.23]
    
    def get_benchmark_scores(targets,df):
        
        # Get standardized predictions
        pred_fields = ["BA","BA_pine","BA_cedar","BA_fir","BA_oak",
                       "TreeStemD", "TreeStemD_40","TreeStemD_60",
                       "TreeStemD_80","TreeStemD_100","Pct_shrub_cover"]
        data = np.array(df[pred_fields])
        std = data.std(axis = 0)
        mean = data.mean(axis = 0)
        data -= mean
        data /= std
        
        # Get standardized targets
        std_targets = (np.array(targets) - mean) / std
        
        scores = []
        
        for row in range(data.shape[0]):
            preds = data[row,:]
            biases = std_targets - preds
            scores.append(root_mean_square(biases))
        return scores
    
    # Generate the scores
    scores = get_benchmark_scores(targets,df)
    df['rmse'] = scores
    
    # Filter to members that pass some simple sanity checks on what a pre-colonial forest should look like
    
    # There should be at least 5 tree above 40 cm dbh
    enough_large_trees = df["TreeStemD_40"] > 5
    
    # There should be at least 5 percent total shrub cover
    enough_shrubs = df["Pct_shrub_cover"] > 0.05
    
    # Pine should not be failing
    enough_pine = df["BA_pine"] > 0.1
    
    # There should be no more than 2 pfts failing
    coexistence_filter = df["FailedPFTs"] < 3
    
    # AGB should be at least 2 Kg of Carbon per m2
    enough_agb = df["AGB"] > 2
    
    # All filters
    all_filters = enough_large_trees & enough_shrubs & enough_pine & coexistence_filter & enough_agb
    
    df = df[all_filters]
    print("There are",len(df),"passing parameter sets")
    
    # Add the tag number of the next simulation to the passing ensemble member csv so that it can later be used
    # to match structure metrics from CZ2-PEAS to CZ2-trans
    next_inst_nums = list(range(1,len(df) + 1))
    next_inst_tags = [str(num).rjust(4, '0') for num in next_inst_nums]
    df["CZ2-trans-id"] = [new_subdirs_prefix + "_" + tag for tag in next_inst_tags]
    
    key_metrics.append("rmse")
    
    # Write csv of passing members
    if write_csv_of_passing_members == True:
        df.to_csv(output_file_path)
        print("Wrote structure metrics for param sets that pass CZ2-PEAS:",output_file_path)
    
    if setup_new_simulation == True:
        aggregate_passing_paramsets(successfull_param_files = list(df['param_file_path']),
                                n_new_cases = 1,
                                n_inst_per_case = len(df),
                                new_subdirs_prefix = new_subdirs_prefix, # Without underscore
                                path_to_ensemble_param_files_root = path_to_ensemble_param_files_root,
                                new_param_file_base_name = 'ca_5pfts_100523') # Without underscore

There are 54 passing parameter sets
Wrote structure metrics for param sets that pass CZ2-PEAS: /glade/scratch/adamhb/processed_output/CZ2_equilibrium_passing/CZ2-PEAS_passing_110923.csv
All files in tmp have been deleted.
originl_file: /glade/u/home/adamhb/ahb_params/fates_api_25/ensembles/CZ2_equilibrium_110523_03/CZ2_equilibrium_110523_0058.nc
tmp file: tmp/ca_5pfts_100523_0001.nc
originl_file: /glade/u/home/adamhb/ahb_params/fates_api_25/ensembles/CZ2_equilibrium_110323_02/CZ2_equilibrium_110323_0023.nc
tmp file: tmp/ca_5pfts_100523_0002.nc
originl_file: /glade/u/home/adamhb/ahb_params/fates_api_25/ensembles/CZ2_equilibrium_110523_01/CZ2_equilibrium_110523_0028.nc
tmp file: tmp/ca_5pfts_100523_0003.nc
originl_file: /glade/u/home/adamhb/ahb_params/fates_api_25/ensembles/CZ2_equilibrium_110523_01/CZ2_equilibrium_110523_0010.nc
tmp file: tmp/ca_5pfts_100523_0004.nc
originl_file: /glade/u/home/adamhb/ahb_params/fates_api_25/ensembles/CZ2_equilibrium_110523_01/CZ2_equilibrium_110523_0054

In [348]:
df[['inst_id','CZ2-trans-id']]

,inst_id,CZ2-trans-id
345,CZ2_equilibrium_110523_03_-17e2acb6a_FATES-8a054a12_0058,CZ2_trans_110923_0001
58,CZ2_equilibrium_110323_02_-17e2acb6a_FATES-8a054a12_0023,CZ2_trans_110923_0002
171,CZ2_equilibrium_110523_01_-17e2acb6a_FATES-8a054a12_0028,CZ2_trans_110923_0003
153,CZ2_equilibrium_110523_01_-17e2acb6a_FATES-8a054a12_0010,CZ2_trans_110923_0004
197,CZ2_equilibrium_110523_01_-17e2acb6a_FATES-8a054a12_0054,CZ2_trans_110923_0005
100,CZ2_equilibrium_110323_03_-17e2acb6a_FATES-8a054a12_0029,CZ2_trans_110923_0006
37,CZ2_equilibrium_110323_02_-17e2acb6a_FATES-8a054a12_0002,CZ2_trans_110923_0007
60,CZ2_equilibrium_110323_02_-17e2acb6a_FATES-8a054a12_0025,CZ2_trans_110923_0008
192,CZ2_equilibrium_110523_01_-17e2acb6a_FATES-8a054a12_0049,CZ2_trans_110923_0009
32,CZ2_equilibrium_110323_01_-17e2acb6a_FATES-8a054a12_0033,CZ2_trans_110923_0010


## Compare Transient to PEAS

In [355]:
if check_transient_vs_precolonial == True:
    
    # Criteria
    def get_change_in_metric(scenarioA_data,scenarioB_data,pft_name,metric = "BA_dominance"):
        
        if metric == "BA_dominance":
            pft_ba = "BA_" + pft_name

            scenarioA_frac_of_total = scenarioA_data[pft_ba] / scenarioA_data['BA']
            scenarioB_frac_of_total = scenarioB_data[pft_ba] / scenarioB_data['BA']
        
            response = scenarioB_frac_of_total > scenarioA_frac_of_total
        
        if metric == "Pct_shrub_cover":
            
            response = scenarioB_data["Pct_shrub_cover"] > scenarioA_data["Pct_shrub_cover"]
        
        return response    
    
    
    # The datasets here can be without the parameter values, but needs to have the inst ids
    def pft_response_to_fire_regime(PEAS_data,TRANS_data):
        
        print("Check TRANS fire regime!!")
        
        if TRANS_data['Burned_area'] > 0.01:
            print("TRANS fire regime has more burned area than 1984 to 2009")
        
        if TRANS_data['Pct_high_severity_3500'] > 31:
            print("TRANS fire regime has more high severity fire than 1984 to 2009")
        
        # 1. Increase in fir dominance with fire suppression
        fir_response = get_change_in_metric(PEAS_data,TRANS_data,"fir")
        
        # 2. Increase in cedar dominance with fire suppression
        cedar_response = get_change_in_metric(PEAS_data,TRANS_data,"cedar")
        
        # These next two really depend on the state of the TRANS fire regime
        # are we in a situation where burned area is coming back up?
        # are we in a situation where fire intensity is coming back up?
        
        # 3. Higher in pine dominance during PEAS
        pine_response = get_change_in_metric(TRANS_data,PEAS_data,"pine")
        
        # 4. Higher oak dominance during PEAS
        oak_response = get_change_in_metric(TRANS_data,PEAS_data,"oak")
        
        # 5. Lower shrub cover during fire suppression
        shrub_response = get_change_in_metric(PEAS_data,TRANS_data,"shrub",metric="Pct_shrub_cover")
        
        pft_response_score = sum(fir_response,cedar_response,pine_response,oak_response,shrub_response) / 5
        
        return pft_response_score

## Compare CZ2 to STAN

Probably faster to do this in the transient state because running a transient simulation is faster than equilibrium

In [358]:
if check_CZ2_vs_STAN == True:
    # The datasets here can be without the parameter values, but needs to have the inst ids
    def pft_response_to_climate(dry_data,wet_data):
        
        # 1. Increase in fir dominance at wet site
        fir_response = get_change_in_metric(dry_data,wet_data,"fir")
        
        # 2. Increase in shrub and oak dominance at warm dry site
        oak_response = get_change_in_metric(wet_data,dry_data,"oak")
        
        # 3. Increase in shrub dominance at warm dry site
        shrub_response = get_change_in_metric(wet_data,dry_data,"shrub",metric="Pct_shrub_cover")
        
        pft_response_score = sum(fir_response,oak_response,shrub_response) / 3
        
        return pft_response_score

## Transfer restart files to new case

In [352]:
if new_run_is_hybrid == True:

    def extract_substring(s: str) -> str:
        # This regular expression looks for a sequence of characters that are
        # after "ensembles/" and followed by a "/", without including the "/"
        match = re.search(r'ensembles/([^/]*)', s)
        if match:
            # The substring is in the first capturing group
            return match.group(1)
        else:
            # You can change this to return None or an empty string if preferred
            return "No match found"

    def get_case_from_reference_param_file(reference_param_file):
        prefix = extract_substring(reference_param_file)
        return prefix + '_-17e2acb6a_FATES-8a054a12'

    def find_nc_and_rpointer_files(directory):
        matches = []
        for root, dirnames, filenames in os.walk(directory):
            for filename in fnmatch.filter(filenames, '*.nc'):
                matches.append(os.path.join(root, filename))
            for filename in filenames:
                if "rpointer" in filename:
                    matches.append(os.path.join(root, filename))
        return matches

    def extract_number_from_filename(filename):
        pattern = re.compile(r'(\d{4})\.nc$')
        match = pattern.search(filename)
        if match:
            return match.group(1)
        else:
            return None

    def replace_number_in_filename(filename, new_number):
        # Check if the new number is exactly four digits
        if not re.match(r'^\d{4}$', new_number):
            raise ValueError("The new number must be a four-digit string.")

        # Regular expression to find four-digit numbers between an "_" and "." or at the end of the string
        pattern = re.compile(r'(?<=_)(\d{4})(?=\.|$)')

        # Find all matches
        matches = pattern.findall(filename)

        # Raise an error if there are more than two matches
        if len(matches) > 2:
            raise ValueError("The filename contains more than two four-digit numbers, cannot proceed.")

        # Replace the first occurrence of a four-digit number after "_" and before "." or at the end of the string
        new_filename = pattern.sub(new_number, filename, count=1)

        return new_filename

    def replace_two_digit_number(filename):
        # Regular expression to match a two-digit number between two underscores
        pattern = re.compile(r'(?<=_)\d{2}(?=_)')

        # Replace the two-digit number with "01"
        new_filename = pattern.sub("01", filename)

        return new_filename
    
    def replace_before_dot_with_string(original_string, replacement, keyword="FATES"):
        # Check if the keyword "FATES" is in the original string
        if keyword in original_string:
            # Split the original string into two parts at the first dot
            parts = original_string.split('.', 1)
            # Check if there's at least one dot to split on
            if len(parts) > 1:
                # Replace the part before the first dot with the user-defined string
                return replacement + '.' + parts[1]
        # If the keyword isn't found, or there's no dot, return the original string
        return original_string



    def transfer_restart_files_to_new_case(ref_param_file,harmonized_reference_case_name,
                                           destination_run_dir = None,
                                           destination_inst_tag = None,
                                           manual_case_name = None):
        ref = ref_param_file
        if manual_case_name != None:
            case_name = manual_case_name
        else:
            case_name = get_case_from_reference_param_file(ref)
        case_rundir = os.path.join('/glade/scratch/adamhb/archive',case_name,'run')
        ref_inst_tag = extract_number_from_filename(ref)
        all_nc_and_pointer_files = find_nc_and_rpointer_files(case_rundir)
        matching_files = [f for f in all_nc_and_pointer_files if ref_inst_tag in f]
        for i in matching_files:
            new_file_name = replace_before_dot_with_string(os.path.basename(replace_number_in_filename(i,destination_inst_tag)),harmonized_reference_case_name)
            new_full_file_path = os.path.join(destination_run_dir,new_file_name)
            print("Copying",i,"to",new_full_file_path)
            shutil.copy(i,new_full_file_path)
        

    def transfer_all_restart_files_to_new_case(ref_param_files,
                                               harmonized_reference_case_name,
                                               destination_run_dir = None,
                                               manual_case_name = None):
        inst_nums = list(range(1,len(ref_param_files) + 1))
        print(len(inst_nums))
        for i,ref in enumerate(ref_param_files):
            new_tag = str(inst_nums[i]).rjust(4, '0')
            transfer_restart_files_to_new_case(ref,harmonized_reference_case_name,destination_run_dir = destination_run_dir,destination_inst_tag = new_tag) 

### Execute

In [346]:
if (new_run_is_hybrid == True) & (setup_new_simulation == True):
    
    param_files = os.listdir('/glade/u/home/adamhb/ahb_params/fates_api_25/ensembles/CZ2_trans_110923_01/')
    
    transfer_all_restart_files_to_new_case(ref_param_files = param_files,
                                           harmonized_reference_case_name=harmonized_reference_case_name, # This is arbitrarily on the of the cases contributing to the successful CZ2-PEAS param sets
                                           destination_run_dir = destination_run_dir,
                                           manual_case_name='CZ2_trans_110923_-17e2acb6a_FATES-8a054a12')

54
Copying /glade/scratch/adamhb/archive/CZ2_equilibrium_110523_03_-17e2acb6a_FATES-8a054a12/run/CZ2_equilibrium_110523_03_-17e2acb6a_FATES-8a054a12.cpl_0058.r.2300-01-01-00000.nc to /glade/scratch/adamhb/archive/CZ2_trans_110923_-17e2acb6a_FATES-8a054a12/run/CZ2_equilibrium_110523_01_-17e2acb6a_FATES-8a054a12.cpl_0001.r.2300-01-01-00000.nc
Copying /glade/scratch/adamhb/archive/CZ2_equilibrium_110523_03_-17e2acb6a_FATES-8a054a12/run/CZ2_equilibrium_110523_03_-17e2acb6a_FATES-8a054a12.clm2_0058.r.2300-01-01-00000.nc to /glade/scratch/adamhb/archive/CZ2_trans_110923_-17e2acb6a_FATES-8a054a12/run/CZ2_equilibrium_110523_01_-17e2acb6a_FATES-8a054a12.clm2_0001.r.2300-01-01-00000.nc
Copying /glade/scratch/adamhb/archive/CZ2_equilibrium_110523_03_-17e2acb6a_FATES-8a054a12/run/CZ2_equilibrium_110523_03_-17e2acb6a_FATES-8a054a12.clm2_0058.h0.2299-12.nc to /glade/scratch/adamhb/archive/CZ2_trans_110923_-17e2acb6a_FATES-8a054a12/run/CZ2_equilibrium_110523_01_-17e2acb6a_FATES-8a054a12.clm2_0001.h0.